In [ ]:
addprocs(11)

In [1]:
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in eval at ./sysimg.jl:14
 in anonymous at multi.jl:1350
 in run_work_thunk at multi.jl:645
 in remotecall_fetch at multi.jl:718
 in remotecall_fetch at multi.jl:734
 in anonymous at multi.jl:1352
while loading /home/ubuntu/.julia/v0.5/FunctionalCollections/src/PersistentArrayMap.jl, in expression starting on line 58
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in require at ./loading.jl:243
 in eval at ./sysimg.jl:14
 in anonymous at multi.jl:1350
 in run_work_thunk at multi.jl:645
 in remotecall_fetch at multi.jl:718
 in remotecall_fetch at multi.jl:734
 in anonymous at multi.jl:1352
while lo

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [5]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere data_filename = "atis_data2.jsz"
#@everywhere data_filename = "doctors_letters.jsz"
@everywhere data = open(data_filename,"r") do fh
    deserialize(fh)    
end
#res_file = "atis_res_glove.jsz"
#res_file = "doctors_letters_res_glove.jsz"
unordered_output, test_set = open(res_file,"r") do fh
    deserialize(fh)    
end

#data = open("atis_data2.jsz","r") do fh
#    deserialize(fh)    
#end

#const zeroed_words = data["zeroed_words"]

@everywhere const zeroed_words = ASCIIString[]


LoadError: LoadError: UndefVarError: res_file not defined
while loading In[5], in expression starting on line 13

In [6]:
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere  @pyimport nltk.probability as nltk_prob


@everywhere kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, data["trigrams"]))
@everywhere function trigram_model(given1::AbstractString, given2::AbstractString, event::AbstractString)
    kn_prob_dist[:prob]((given1, given2, event))
end

In [70]:
function sname(cur_state, remaining_words)
    @pipe [cur_state[1], cur_state[2],remaining_words...] |> map(x->x[1],_) |> join
end

sname (generic function with 1 method)

In [174]:
@everywhere const log=false
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

@everywhere function transition_prob(cur_state::State{S}, next_word::S)
    trigram_model(cur_state..., next_word)
end


@everywhere _get_options_cache_type = Dict{Tuple{State{S}, Any}, Vector{Tuple{plist{S}, Float64}}}
@everywhere _get_options_caches = Dict{Vector{S}, _get_options_cache_type}()

#"""
#returns all possible orderings of the remaining words.
#the freewords are also inserted into every possible position, EXCEPT at the end.
#Freewords also will not be inserted after other freewords
#(For now)
#"""
@everywhere function get_all_orders(free_words::Vector{S}, unordered_words::Vector{S}; ϵ=0.0)
    _get_options_cache = get!(_get_options_cache_type, _get_options_caches, free_words)
    #different cache if different freewords
    
    function get_options(cur_state::State{S})
        if log
            csname = sname(cur_state,[])
            println("\\state{$csname}{$(cur_state[1])}{$(cur_state[2])}{};")

            nsname = sname((cur_state[2], END_MARKER1),[])
            println("\\state{$nsname}{$(cur_state[2])}{$(END_MARKER1)}{};")
            lbltext= "\{E_1\}$(cur_state[1][1])$(cur_state[2][1])"
            println("($csname) ->[edge label=\\p $lbltext] ($nsname);")
        end
        #Can never End in a freeword
        tp = transition_prob(cur_state, END_MARKER1)
        # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
        # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
        [
            (EmptyList{S}(), tp)
        ]
    end
    
    function get_options(state::State{S}, remaining_words::S...)
        get!(_get_options_cache, (state, remaining_words)) do
            _get_options(state, remaining_words...)
        end
    end

    function _get_options_inner(cur_state, word, remaining_words, get_option_fun)
        tp = transition_prob(cur_state, word) 
        next_state = (cur_state[2], word)
        
        #if tp<=-ϵ return end

        for (tail, next_tp) in get_option_fun(next_state, remaining_words...)
            produce(cons(word, tail),tp*next_tp)
        end
    end

    function _get_options_nofree(cur_state::State{S}, remaining_words::S...)
        (@task begin        
            for ii in 1:length(remaining_words)
                word = remaining_words[ii]
                new_remaining_words = remaining_words[[1:ii-1; ii+1:end]]
                
                if log
                    csname = sname(cur_state,remaining_words)
                    nsname = sname((cur_state[2], word),new_remaining_words)
                    lbltext= "$(word[1])$(cur_state[1][1])$(cur_state[2][1])"
                    println("($csname) ->[edge label=\\p $lbltext] ($nsname);")
                end
                
                _get_options_inner(cur_state, word, new_remaining_words, get_options)
            end
        end)
    end
    
    function _get_options_free(cur_state::State{S}, remaining_words::S...)
        @task begin        
            for word in free_words
                #Important: get_options_free, only chains to _get_options_nofree
                _get_options_inner(cur_state, word, remaining_words, _get_options_nofree)
            end
        end
    end
    
    function _get_options(cur_state::State{S}, remaining_words::S...)
        csname = sname(cur_state,remaining_words)
        log && println("\\state{$csname}{$(cur_state[1])}{$(cur_state[2])}{$(join(remaining_words,", "))};")
        chain(_get_options_nofree(cur_state, remaining_words...),
             _get_options_free(cur_state, remaining_words...)) |> collect
    end
   
    initial_state = (START_MARKER1, START_MARKER2)
    get_options(initial_state, unordered_words...)
end

In [183]:
@everywhere function order(unordered_words::Vector{S}, free_words=S[]::Vector{S}; ϵ=0.0::Float64, best_n=1::Int)
    orders_and_probs = get_all_orders(free_words, unordered_words, ϵ=0.0)
    best_n = min(best_n, length(orders_and_probs))
    if best_n==0 #None found
        #warn("No possible Orderding found. Defaulting to unordered: ".*string(unordered_words))
        Tuple[(unordered_words, 0.0)]
    elseif best_n==1 #4x as fast as the else
        max_prob, max_ii = @pipe orders_and_probs |> map(op->op[2],_) |> findmax
        Tuple[orders_and_probs[max_ii]]
    else
        select!(orders_and_probs, 1:best_n, by=op->-op[2] )
    end
end

function norm_order(unordered_words::Vector{S}, free_words=S[]::Vector{S}; ϵ=0.0::Float64, best_n=1::Int)
    orders_and_probs = order(unordered_words, free_words, ϵ=ϵ, best_n=best_n)
    total_prob = @pipe orders_and_probs |> map(op->op[2], _ ) |> sum
    total_prob = total_prob == 0.0 ? 1.0 : total_prob
    [(join(order, " "), prob/total_prob) for (order,prob) in orders_and_probs]
end

norm_order (generic function with 2 methods)

In [196]:
#order(["the", "cheapest","show", "all", "flights", "me"], best_n=500000)
for ii in norm_order(["cheapest","the", "me", "show", "all", "flights"], best_n=500000)
    println("\\item $(ii[1]) \$P=$(ii[2])\$")
end

\item show me all the cheapest flights $P=0.9976611556180363$
\item all flights show me the cheapest $P=0.002287767676692731$
\item flights show me all the cheapest $P=5.099115447705249e-5$
\item cheapest flights show me all the $P=5.0836007018696596e-8$
\item all the cheapest flights show me $P=3.4714786951415333e-8$
\item show flights all cheapest the me $P=0.0$
\item show flights all the cheapest me $P=0.0$
\item show flights all the me cheapest $P=0.0$
\item show flights all me cheapest the $P=0.0$
\item show flights all me the cheapest $P=0.0$
\item all cheapest the me show flights $P=0.0$
\item all cheapest the me flights show $P=0.0$
\item all cheapest the show me flights $P=0.0$
\item all cheapest the show flights me $P=0.0$
\item all cheapest the flights me show $P=0.0$
\item all cheapest the flights show me $P=0.0$
\item all cheapest me the show flights $P=0.0$
\item all cheapest me the flights show $P=0.0$
\item all cheapest me show the flights $P=0.0$
\item all cheapest me 

In [168]:
examples = Any[["to","to","atlanta","pittsburgh","back","like","denver","fly","mealtime","respectively","ask","it","well"],
["to","i","to","atlanta","from","would","denver","fly","like","pittsburgh","were"],
["to","to","atlanta","fly","from","pittsburgh","like","would","and","denver","really","regarding"],
["to","i","to","atlanta","from","denver","fly","would","pittsburgh","wonder","kind","and"],
["to","to","atlanta","pittsburgh","back","denver","and","fly","of","would","i","preferably"],
["to","to","from","pittsburgh","atlanta","fly","and","denver","soon","i","other","entire"],
["to","i","to","atlanta","pittsburgh","and","denver","big","from","liz","apply","flying"],
["to","to","atlanta","from","denver","fly","and","pittsburgh","specials","bound","i","soon","think"],
["to","to","like","atlanta","from","denver","would","fly","pittsburgh","i","both"],
["to","we","to","atlanta","back","fly","pittsburgh","like","including","denver","was","choose"]]

10-element Array{Any,1}:
 ASCIIString["to","to","atlanta","pittsburgh","back","like","denver","fly","mealtime","respectively","ask","it","well"]
 ASCIIString["to","i","to","atlanta","from","would","denver","fly","like","pittsburgh","were"]                         
 ASCIIString["to","to","atlanta","fly","from","pittsburgh","like","would","and","denver","really","regarding"]         
 ASCIIString["to","i","to","atlanta","from","denver","fly","would","pittsburgh","wonder","kind","and"]                 
 ASCIIString["to","to","atlanta","pittsburgh","back","denver","and","fly","of","would","i","preferably"]               
 ASCIIString["to","to","from","pittsburgh","atlanta","fly","and","denver","soon","i","other","entire"]                 
 ASCIIString["to","i","to","atlanta","pittsburgh","and","denver","big","from","liz","apply","flying"]                  
 ASCIIString["to","to","atlanta","from","denver","fly","and","pittsburgh","specials","bound","i","soon","think"]       
 ASCIIString["t

In [172]:
map(examples) do unordered_words
    order(unordered_words, ASCIIString[])
    end

10-element Array{Any,1}:
 Tuple[(ASCIIString["to","to","atlanta","pittsburgh","back","like","denver","fly","mealtime","respectively","ask","it","well"],0.0)]
 Tuple[(ASCIIString["to","i","to","atlanta","from","would","denver","fly","like","pittsburgh","were"],0.0)]                         
 Tuple[(ASCIIString["to","to","atlanta","fly","from","pittsburgh","like","would","and","denver","really","regarding"],0.0)]         
 Tuple[(ASCIIString["to","i","to","atlanta","from","denver","fly","would","pittsburgh","wonder","kind","and"],0.0)]                 
 Tuple[(ASCIIString(of, atlanta, would, i, fly, to, pittsburgh, back, to, denver, and, preferably),2.1693420467360814e-24)]         
 Tuple[(ASCIIString["to","to","from","pittsburgh","atlanta","fly","and","denver","soon","i","other","entire"],0.0)]                 
 Tuple[(ASCIIString["to","i","to","atlanta","pittsburgh","and","denver","big","from","liz","apply","flying"],0.0)]                  
 Tuple[(ASCIIString["to","to","atlanta","fro

In [ ]:
short_cases = Bool[length(ws) <=20 for ws in unordered_output]

true_ordered_sents = test_set[short_cases]
ordered_sents_and_probs = pmap(unordered_output[short_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end

In [ ]:
ordered_sents = map(op->op[1], ordered_sents_and_probs)

In [ ]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    #println("$ii - $match")
end
mean(perfect_matches)

In [ ]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(Any[reference],candidate, weights)
    end
end

In [ ]:
map(bleu_score, ordered_sents,true_ordered_sents) |> mean


In [ ]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

In [ ]:
true_ordered_sents

In [ ]:
ordered_sents

In [ ]:
ordered_sents[50]

In [ ]:
unordered_output[short_cases][eval_cases][50]

In [ ]:
true_ordered_sents[50]

In [ ]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

In [ ]:
Any[true_ordered_sents[1]]


In [ ]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk